In [2]:
# torch, autograd and backpropagation
# https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/


import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

L = 10 - d

print("The grad fn for a is", a.grad_fn)
print("The grad fn for d is", d.grad_fn)

The grad fn for a is None
The grad fn for d is <AddBackward0 object at 0x7f9df81cdb20>


In [3]:
b.is_leaf

False

In [4]:
w1.is_leaf

True

In [5]:
b.grad_fn

In [6]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [7]:
# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print('Hello Im a tensor backward hook reporting on x:', x))  

b.mean().backward() 


print(a.grad, b.grad)

Hello Im a tensor backward hook reporting on x: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [8]:
# Playing with hooks (https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/)

import torch 
import torch.nn as nn

class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
   
  
  def forward(self, x):
    x = self.relu(self.conv(x))
    x.register_hook(lambda grad : torch.clamp(grad, min = 0))     #No gradient shall be backpropagated 
                                                                  #conv outside less than 0
      
    # print whether there is any negative grad
    x.register_hook(lambda grad: print("Gradients less than zero:", bool((grad < 0).any())))  
    return self.fc1(self.flatten(x))
  

net = myNet()

for name, param in net.named_parameters():
  # if the param is from a linear and is a bias
  if "fc" in name and "bias" in name:
    param.register_hook(lambda grad: torch.zeros(grad.shape))


out = net(torch.randn(1,3,8,8)) 

(1 - out).mean().backward()

print("The biases are", net.fc1.bias.grad)     #bias grads are zero

Gradients less than zero: False
The biases are tensor([0., 0., 0., 0., 0.])


In [9]:
# The Forward Hook for Visualising Activations 

visualisation = {}

inp = torch.randn(1,3,8,8)

def hook_fn(m, i, o):
  visualisation[m] = o 
  
net = myNet()

for name, layer in net._modules.items():
  print('net._modules.items().__class__.__name__:', net._modules.items())
  layer.register_forward_hook(hook_fn)
  
out = net(inp) 

tensor([[[[ 1.7005, -1.9498, -0.4868, -0.0303, -0.3445,  0.2933, -0.8732,
           -0.6807],
          [-2.0932, -0.9393, -0.1748,  0.4936,  1.1228, -0.6647, -1.7640,
           -1.0301],
          [ 0.2753,  1.0046,  0.5040, -0.7411,  0.1099, -0.0484, -0.4029,
            2.0737],
          [-1.4201, -0.7080,  0.0212, -1.2915, -0.4933,  1.1316, -0.9295,
            0.5645],
          [-1.9732,  0.2441, -0.3288, -0.7064,  0.5568,  0.7822,  0.1295,
            0.8211],
          [-0.7719,  0.7438, -0.2226,  0.0795,  0.3837,  0.1072, -0.7230,
            0.2285],
          [-1.1002, -0.1511,  0.4822,  0.7264,  0.0208,  1.1334,  1.2841,
            1.5398],
          [ 0.4773, -0.1455,  0.7014,  1.7694,  1.2766, -0.1334, -2.1647,
           -0.6898]],

         [[-0.1906,  1.5169, -0.2449, -1.3430,  2.3499,  0.7428, -0.0139,
           -1.3244],
          [ 0.0750,  1.3354, -0.3667,  1.1475, -0.8353, -1.9871,  1.2343,
            1.0611],
          [ 0.0372, -0.4763,  1.8792, -0.1377, -